<a href="https://colab.research.google.com/github/rhedsantiago/Gradient_Descent/blob/main/Assignment1Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import pandas as pd
import numpy as np
import random
import math

# Class to implement the gradient function at the end
class GradientFunction():

  # Constructor - takes in the dataframe of either training or testing data
  # If you are taking in testing data, you only have to use normalize_variables and gradient_descent function

  def __init__(self, df):
    self.cars = df
    self.bias = 0.0
    self.weights = []
    self.X_vars = pd.DataFrame
    self.Y = pd.DataFrame
    self.Y_hat = np.array([])
    self.updated_weights = []
    self.log = {}

  # normalize our variables
  def normalize_variables(self):
    self.X_vars= self.cars[['cylinders', 'displacement', 'horsepower', 'model_year']]
    self.Y = self.cars['mpg']
    self.Y = np.array((self.Y - self.Y.mean())/self.Y.std())                        #<-----Normalizing the data
    self.X_vars = self.X_vars.apply(lambda rec:(rec-rec.mean())/rec.std(), axis=0)

  # start off with any weights, choose randomly
  def choose_random_weights(self, randomizer):
    self.bias = random.random()
    self.weights = np.random.rand(randomizer)

  # get the estimated y function based off of the random weights from earlier
  def predict_y_func(self):
    self.Y_hat = self.bias + np.dot(self.X_vars, self.weights)

  # calculate the cost error function, used for testing. We want this as close to 0 as possible
  def calculate_cost(self):
    Y_residual = self.Y - self.Y_hat #actual - predicted
    self.predict_y_func()
    return np.sum(np.dot(Y_residual.T, Y_residual))/len(self.Y-Y_residual)

  # function to update the weights from the previously randomly selected ones
  def update_weights(self, learning_rate):
    count = 0

    # keep performing this algorithm until our cost function is less than 0.5
    # this way, we get the cost function as close to 0 as possible

    log_file = open("weights_tested_results.txt", "w")

    while (self.calculate_cost() > 0.5):
      self.predict_y_func()
      self.bias=self.bias-learning_rate*(np.sum(self.Y_hat-self.Y)*2)/len(self.Y)
      self.weights=self.weights-learning_rate*(np.dot((self.Y_hat-self.Y),self.X_vars)*2)/len(self.Y)
      self.calculate_cost()
      count += 1
      key = 'trial ' + str(count)
      tracker = []
      tracker.append(self.bias)
      tracker.append(self.weights[0])
      tracker.append(self.weights[1])
      tracker.append(self.weights[2])
      tracker.append(self.weights[3])
      self.log[key] = tracker


      log_file.write("Trial #: " )
      log_file.write(str(key) + "\n")
      log_file.write("Weights: ")
      log_file.write(str(tracker) + "\n")
      log_file.write('------------------------------------------------------------------------------------------------------------------------------------\n')

  # combine the bias and weights into one array
  def adjust_values(self):
    self.updated_weights.append(self.bias)
    self.updated_weights.append(self.weights[0])
    self.updated_weights.append(self.weights[1])
    self.updated_weights.append(self.weights[2])
    self.updated_weights.append(self.weights[3])

  # Function using residual that will be plugged into the gradient_descent function
  def my_gradient(self):
      res = self.updated_weights[0] + (np.dot(self.updated_weights[1],self.X_vars['cylinders']) - self.Y) + (np.dot(self.updated_weights[2],self.X_vars['displacement']) - self.Y ) + (np.dot(self.updated_weights[3],self.X_vars['horsepower']) - self.Y) + (np.dot(self.updated_weights[4],self.X_vars['model_year']) - self.Y)
      #return res.mean(), (np.dot(res, x)).mean()  # .mean() is a method of np.ndarray
      return res.mean(), (np.dot(res, self.X_vars['cylinders'])).mean(), (np.dot(res, self.X_vars['displacement'])).mean(), (np.dot(res, self.X_vars['horsepower'])).mean(), (np.dot(res, self.X_vars['model_year'])).mean()
      # for vector of x (res * x[0]).mean(), res (res*x[1]).mean()

  # Gradient descent function from class
  def gradient_descent(self, learn_rate=0.1, n_iter=50, tolerance=1e-06):
    vector = self.updated_weights
    for _ in range(n_iter):
      diff = -learn_rate * np.array(self.my_gradient())
      if np.all(np.abs(diff) <= tolerance):
        break
      vector += diff
    return vector

cars = pd.read_csv('https://raw.githubusercontent.com/Jerpac/CS4375/main/auto-mpg.data', delim_whitespace = True)

# Pre-processing the data ----------------------
# this entire thing will be separate from the class?

cars.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']

cars = cars.dropna()
cars = cars.drop_duplicates()
cars = cars.drop(columns = ['car_name', 'origin', 'weight', 'acceleration'])
cars = cars.drop(cars[cars['horsepower'] == '?'].index)

cars["cylinders"] = pd.to_numeric(cars["cylinders"], downcast="integer")
cars["displacement"] = pd.to_numeric(cars["displacement"], downcast="float")
cars["horsepower"] = pd.to_numeric(cars["horsepower"], downcast="float")
cars["model_year"] = pd.to_numeric(cars["model_year"], downcast="integer")

cars = cars.reset_index(drop=True)

training_data = cars.sample(frac = 0.8, random_state = 100)
test_data = cars.drop(training_data.index)
# Create the class
car_gradient = GradientFunction(training_data)
# Normalize our variables
car_gradient.normalize_variables()
# get random values for bias and weights
car_gradient.choose_random_weights(4)
# Calculate the output of Y
car_gradient.predict_y_func()
# Calculate the cost function (just to see where we are)
car_gradient.calculate_cost()
# Update the weights for our cost function
car_gradient.update_weights(0.01)
# Adjust the values
car_gradient.adjust_values()
# Get my gradient
car_gradient.my_gradient()
# Call the gradient descent algorithm
print(f'Result of Gradient Descent Function Using Training Data: {car_gradient.gradient_descent(0.0001, 1)}')

weights = car_gradient.updated_weights
car_gradient2 = GradientFunction(test_data)
car_gradient2.updated_weights = weights
car_gradient2.normalize_variables()

print(f'Result of Gradient Descent Function Using Test Data: {car_gradient2.gradient_descent(0.0001, 1)}')
print(f'Weights used for both: {weights}')

Result of Gradient Descent Function Using Training Data: [ 0.19920926 -0.18274627 -0.39671266  0.0198157   0.83586108]
Result of Gradient Descent Function Using Test Data: [ 0.19920926 -0.1244862  -0.33590638  0.07625672  0.79950752]
Weights used for both: [0.19922918226731534, -0.1046499718165445, -0.31571513515044647, 0.09682568170532603, 0.7900747096872023]
